# GSoC pretest

## Imports

In [12]:
import torch
import timm
import torch.nn as nn
from PIL import Image
from torchvision import transforms
import torch.quantization as quantization
import matplotlib.pyplot as plt
from openvino.runtime import Core
from openvino.runtime import serialize
from openvino.tools import mo
import onnx
import onnxruntime as ort
from onnxruntime.quantization import QuantType, quantize_dynamic

## Load Model

In [2]:
model_names = timm.list_models('swin_small*')
for model_name in model_names:
    print(model_name)
model = timm.create_model('swin_small_patch4_window7_224', pretrained=True)

swin_small_patch4_window7_224


/home/user/miniconda3/envs/openvino_env/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


## Run model at ont-quantization

In [16]:
# Load image and preprocess
image = Image.open('../notebooks/data/image/coco.jpg')
# plt.imshow(image)
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.299, 0.224, 0.255])
])
image = transform(image).unsqueeze(0)

# Predict the class of image
with torch.no_grad():
    output = model(image)
    pred = output.argmax(dim=1).item()
    print(f'Predicted class: {pred}')
# After searching, the 208th category in ImageNet is dog.

Predicted class: 208


## Define convert model to onnx and IR function

In [4]:
def convert_models(model, model_input, path):
    script_model = torch.jit.trace(model, model_input)
    torch.onnx.export(script_model,  model_input, path)   
    convert_model = mo.convert_model(path)
    # Change the .onnx suffix to .xml
    IR_path = path[:-4]+'xml'
    serialize(convert_model, IR_path)

## Convert model and test them

In [5]:
# Convert model to onnx and IR
# If you already have a models directory, place comment the next line of code.
!mkdir models
model.eval()
onnx_path = 'models/swin_small_patch4_window7_224.onnx'
model_input = torch.randn(1,3,224,224).cpu() 
# convert_models(model, model_input,onnx_path)

mkdir: 无法创建目录"models": 文件已存在


## Use Pytorch built-in quantization to quantize the model and convert them

In [6]:
# ie = Core()
# model_ir = ie.read_model(model=(onnx_path[:-4]+'xml'))
# model.qconfig = torch.quantization.get_default_qconfig('fbgemm')
model_int8 = quantization.quantize_dynamic(model, dtype=torch.qint8)
# quantization.prepare(model, inplace=True)
# model_int8 = quantization.convert(model, inplace=True)
with torch.no_grad():
    output = model_int8(image)
    pred = output.argmax(dim=1).item()
    print(f'Predicted class: {pred}')
# torch.save(model_int8, 'models/quantize_swin.pth')

Predicted class: 208


In [10]:
# model_onnx = onnx.load(onnx_path)
quantize_path = 'models/quantize_swin.onnx'
quantize_dynamic(model_input=onnx_path,
    model_output= quantize_path,
    weight_type=QuantType.QInt8,
    optimize_model=True
)


Ignore MatMul due to non constant B: /[/layers/layers.0/blocks/blocks.0/attn/MatMul]
Ignore MatMul due to non constant B: /[/layers/layers.0/blocks/blocks.0/attn/MatMul_1]
Ignore MatMul due to non constant B: /[/layers/layers.0/blocks/blocks.1/attn/MatMul]
Ignore MatMul due to non constant B: /[/layers/layers.0/blocks/blocks.1/attn/MatMul_1]
Ignore MatMul due to non constant B: /[/layers/layers.1/blocks/blocks.0/attn/MatMul]
Ignore MatMul due to non constant B: /[/layers/layers.1/blocks/blocks.0/attn/MatMul_1]
Ignore MatMul due to non constant B: /[/layers/layers.1/blocks/blocks.1/attn/MatMul]
Ignore MatMul due to non constant B: /[/layers/layers.1/blocks/blocks.1/attn/MatMul_1]
Ignore MatMul due to non constant B: /[/layers/layers.2/blocks/blocks.0/attn/MatMul]
Ignore MatMul due to non constant B: /[/layers/layers.2/blocks/blocks.0/attn/MatMul_1]
Ignore MatMul due to non constant B: /[/layers/layers.2/blocks/blocks.1/attn/MatMul]
Ignore MatMul due to non constant B: /[/layers/layers.2

NotImplemented: [ONNXRuntimeError] : 9 : NOT_IMPLEMENTED : Could not find an implementation for ConvInteger(10) node with name '/patch_embed/proj/Conv_quant'

In [15]:
onnx_model = onnx.load(onnx_path)
onnx.checker.check_model(onnx_model, full_check=True)

In [8]:
convert_model = mo.convert_model(quantize_path)
# Change the .onnx suffix to .xml
IR_path = quantize_path[:-4]+'xml'
serialize(convert_model, IR_path)
# convert_models(torch_model,model_input, quantize_path)


In [ ]:
model_int8.eval()
quantize_path = 'models/quantize_swin.onnx'
# convert_models(model_int8,model_input, quantize_path)
# script_model = torch.jit.trace(model_int8, model_input)
# torch.onnx.export(script_model,  model_input, quantize_path)
# convert_model = mo.convert_model(quantize_path)
# # # Change the .onnx suffix to .xml
# IR_path = quantize_path[:-4]+'xml'
# serialize(convert_model, IR_path)